# 語意核心

在此範例程式碼中，您將使用 [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI 框架來建立一個基本代理。

此範例的目標是向您展示在後續範例程式碼中實現不同代理模式時所需的步驟。


## 匯入所需的 Python 套件


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## 建立客戶端

在此範例中，我們將使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 來存取 LLM。

`ai_model_id` 被定義為 `gpt-4o-mini`。嘗試將模型更改為 GitHub Models 市場中其他可用的模型，看看不同的結果。

為了使用 `Azure Inference SDK`（用於 GitHub Models 的 `base_url`），我們將在 Semantic Kernel 中使用 `OpenAIChatCompletion` 連接器。此外，還有其他 [可用的連接器](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion)，可以用來在其他模型提供者中使用 Semantic Kernel。


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## 建立代理

以下我們建立名為 `TravelAgent` 的代理。

在這個範例中，我們使用非常簡單的指令。您可以更改這些指令，看看代理如何產生不同的回應。


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## 執行代理

現在我們可以透過定義一個類型為 `ChatHistoryAgentThread` 的執行緒來執行代理。任何必要的系統訊息都可以透過 `invoke_stream` 的 `messages` 關鍵字參數提供給代理。

在定義這些之後，我們會建立一個 `user_inputs`，這是用戶發送給代理的內容。在這個例子中，我們將訊息設為 `Plan me a sunny vacation`。

您可以隨意更改這個訊息，看看代理會有什麼不同的回應。


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，請注意自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
